In [1]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd


# DataFrame 출력 최대 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# 상세도로망 기준으로 교통사고 발생 도로 뽑아내기

In [2]:
df = gpd.read_file('2.대전광역시_교통사고격자(2017~2019).geojson', driver= 'GeoJSON')
df

,gid,acci_cnt,geometry
0,다바866110,0,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,다바823157,0,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,다바928260,0,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,다바931203,2,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,다바836109,0,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."
...,...,...,...
54907,다바861065,0,"MULTIPOLYGON (((127.34527 36.25565, 127.34527 ..."
54908,다바912036,0,"MULTIPOLYGON (((127.40208 36.22956, 127.40207 ..."
54909,다바867189,0,"MULTIPOLYGON (((127.35174 36.36745, 127.35174 ..."
54910,다바851062,0,"MULTIPOLYGON (((127.33415 36.25293, 127.33415 ..."


In [3]:
df2 = gpd.read_file('19.대전광역시_상세도로망(2018).geojson', driver= 'GeoJSON')
df2

,link_id,max_speed,road_name,road_no,road_rank,link_type,road_type,facil_name,up_lanes,dw_lanes,oneway,length,width,car_lane,num_cross,barrier,up_its_id,dw_its_id,geometry
0,553700043,50,호남고속도로지선,251,108,8,0,None,1,0,1,0.645,1,0,0,0,1850000700,0,"MULTILINESTRING ((127.27979 36.24906, 127.2799..."
1,553700273,30,장안로,0,104,32768,0,None,1,1,0,0.673,2,0,0,0,1850015800,1850015700,"MULTILINESTRING ((127.34376 36.24681, 127.3436..."
2,553700308,30,장안로,0,104,32768,0,None,1,1,0,1.095,2,0,0,0,1850015300,1850015400,"MULTILINESTRING ((127.34510 36.24442, 127.3451..."
3,553700340,0,벌곡로330번길,0,104,34816,0,None,1,1,0,0.065,1,0,0,0,0,0,"MULTILINESTRING ((127.29585 36.22254, 127.2958..."
4,553700456,30,장안로,0,104,32768,0,None,1,1,0,0.276,2,0,0,0,1850015700,1850015800,"MULTILINESTRING ((127.34376 36.24681, 127.3450..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10841,565110608,60,신상로,0,104,32768,0,None,1,1,0,0.265,2,0,0,15,1830045600,1830045500,"MULTILINESTRING ((127.52063 36.35414, 127.5205..."
10842,565110629,70,옥천로,4,103,32768,0,None,2,2,0,1.209,3,0,0,5,1830010501,1830010601,"MULTILINESTRING ((127.50770 36.33864, 127.5048..."
10843,565110680,0,신상로647번길,0,104,32768,0,None,1,1,0,0.047,2,0,0,0,0,0,"MULTILINESTRING ((127.51468 36.35256, 127.5149..."
10844,565110681,60,신상로,0,104,32768,0,None,2,2,0,0.3,3,0,0,15,1830045600,1830045500,"MULTILINESTRING ((127.51787 36.35341, 127.5149..."


# intersects 활용해 두 데이터간 geometry가 교차하는 지점 추출
---
* 데이터 범위가 커서 한번에 시도할 경우 6시간 이상 걸림
* df에 로드한 2번 데이터가 외부 유출이 불가능
* 더욱이, 공모전에 사용된 서버는 불규칙하게 서버가 터지는 상황이 발생
* 이에 index를 기준으로 k값을 매기고, 일정 순번까지 완료되면 파일을 저장하도록함

In [ ]:
k=3000
num=0
while k<len(df2['geometry']):
    gc=[]
    num+=1
    for i in range (k,k+500):
        print('현재',i, '번째 입니다.')
        for j in range (len(df4['geometry'])): #df4 = 도로망
            if df4['geometry'][j].intersects(df2['geometry'][i]): #df2 = 대구시 전체 격자
                gc.append(i)
    gc=set(gc)
    gc=list(gc)
    gc_2=df2.iloc[gc]
    gc_2=gc_2.reset_index()
    filename='도로격짜실전화이팅_윤모'+str(num)+'.json'
    gc_2.to_file(filename, driver="GeoJSON")
    #filename='안돼애애'+str(num)+'.json'
    #gc1.to_json(filename,orient='table')
    print('------------------------------현재', + (k+300),'번째까지 완료 했습니다.--------------------------')
    k+=500
    if num = 20;
    break

# 저장된 파일 불러와서 합치기

In [ ]:
#final=pd.DataFrame(columns=['index','gid','acci_cnt','geometry'])
for i in range(2,15):
    filename = "도로격짜실전화이팅"+str(i)+".json"
    file = open(filename,encoding="utf8")
    df2 = gpd.read_file(file)
    df2 = df2.sort_values('index')
    final=pd.concat([final,df2])

In [ ]:
for i in range(21,23):
    filename = "도로격짜실전화이팅"+str(i)+".json"
    file = open(filename,encoding="utf8")
    df2 = gpd.read_file(file)
    df2 = df2.sort_values('index')
    final=pd.concat([final,df2])

위와 같이 번호순으로 저장된 파일명을 불러와서 병합하는 작업 반복

In [ ]:
# 최종 결과 저장
final.to_file('doro2.json', driver="GeoJSON")